In [1]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, MaxPool2D, Conv2D, Flatten, Dropout, SpatialDropout1D, SimpleRNN, GRU, LSTM
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
import random
import pandas as pd
import time
import pickle
import copy

Using TensorFlow backend.


# Binary Classification (Bach+Mozart)

In [2]:
#reading the training rolls and training labels
training_rolls = []
training_labels = []
for i in range(801):
    filename1 = '/kaggle/input/proll-npy/train_rolls/train_rolls/train_roll' + str(i) + '.npy'
    filename2 = '/kaggle/input/proll-npy/train_labels/train_labels/train_label' + str(i) + '.npy'
    temp1 = np.load(filename1)
    temp2 = np.load(filename2)
    training_rolls.append(temp1)
    training_labels.append(temp2)

In [3]:
#reading the test rolls and test labels
test_rolls = []
test_labels = []
for i in range(201):
    filename1 = '/kaggle/input/proll-npy/test_rolls/test_rolls/test_roll' + str(i) + '.npy'
    filename2 = '/kaggle/input/proll-npy/test_labels/test_labels/test_label' + str(i) + '.npy'
    temp1 = np.load(filename1)
    temp2 = np.load(filename2)
    test_rolls.append(temp1)
    test_labels.append(temp2)

In [4]:
test_rolls = np.array(test_rolls)
test_labels = np.array(test_labels)
training_rolls = np.array(training_rolls)
training_labels = np.array(training_labels)

In [5]:
num_classes = 2

piano_roll_shape = training_rolls[0].shape
piano_roll_size = piano_roll_shape[0]*piano_roll_shape[1]

In [6]:

training_labels = to_categorical(training_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

In [7]:
conv_training_data = training_rolls.reshape(801, piano_roll_shape[0], piano_roll_shape[1], 1)
conv_test_data = test_rolls.reshape(201, piano_roll_shape[0], piano_roll_shape[1], 1)

In [8]:
test_rolls = None
training_rolls = None

In [9]:
model = Sequential()
model.add(Conv2D(input_shape=(piano_roll_shape[0],piano_roll_shape[1],1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(num_classes, activation='softmax'))

In [10]:
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 71, 700, 64)       640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 700, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 350, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 35, 350, 128)      73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 35, 350, 128)      147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 175, 128)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 17, 175, 256)     

In [11]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=7, verbose=1, mode='auto')
hist = model.fit(conv_training_data, training_labels, batch_size=64, epochs=30, verbose=True, validation_split=.1,callbacks=[checkpoint,early])

Train on 720 samples, validate on 81 samples
Epoch 1/30
720/720 [==============================] - 673s 935ms/step - loss: 2.0825 - accuracy: 0.5278 - val_loss: 0.6811 - val_accuracy: 0.5926
Epoch 2/30


/opt/conda/lib/python3.7/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/opt/conda/lib/python3.7/site-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


720/720 [==============================] - 681s 945ms/step - loss: 0.6985 - accuracy: 0.4708 - val_loss: 0.6942 - val_accuracy: 0.4074
Epoch 3/30
720/720 [==============================] - 678s 941ms/step - loss: 0.6936 - accuracy: 0.5264 - val_loss: 0.7059 - val_accuracy: 0.4074
Epoch 4/30
720/720 [==============================] - 689s 957ms/step - loss: 0.6919 - accuracy: 0.5264 - val_loss: 0.7008 - val_accuracy: 0.4074
Epoch 5/30
720/720 [==============================] - 671s 932ms/step - loss: 0.6828 - accuracy: 0.5264 - val_loss: 0.6758 - val_accuracy: 0.6420
Epoch 6/30
720/720 [==============================] - 689s 957ms/step - loss: 0.6749 - accuracy: 0.5208 - val_loss: 0.6760 - val_accuracy: 0.6296
Epoch 7/30
720/720 [==============================] - 677s 941ms/step - loss: 0.6813 - accuracy: 0.5958 - val_loss: 0.6695 - val_accuracy: 0.6543
Epoch 8/30
720/720 [==============================] - 665s 923ms/step - loss: 0.6742 - accuracy: 0.5806 - val_loss: 0.6414 - val_accura

In [12]:
loss, accuracy  = model.evaluate(conv_test_data, test_labels, verbose=True)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')

plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')

plt.show()

print(f'Test loss: {loss:.3}')
print(f'Test accuracy: {accuracy:.3}')

201/201 [==============================] - 50s 250ms/step


NameError: name 'history' is not defined